<a href="https://colab.research.google.com/github/Zewtta/PEL219_2025_RedesNeuraisArtificiais/blob/main/PEL216_CNN_3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tarefa 7
##Aluno: Matheus Vieira Martins
##Mestrado Eng. Mecânica

#### Anotações de aulas:
Faça a classificação de algum problema 3D
Os mais conhecidos:
- MedMNISTv2:
- - O MAIS FACIL DE USAR.
Importadiretono Python.
Diversosdatasets.
https://medmnist.com/
- MosMedData (3,7 Gb):
- - https://www.kaggle.com/datasets/andrewmvd/mosmed-covid19-ct scans
- LIDC-IDRI (133 Gb):
- - https://www.cancerimagingarchive.net/collection/lidc-idri/

Tarefa
1. Pode fazer classificação de atividades de vídeo?
2. Pode, mas os tamanhos dos datasets são um obstáculo.
3. Fiquem a vontade para tentar se quizerem.

#Loading dos dados

In [5]:
!pip install medmnist tensorflow numpy scikit-learn

In [6]:
import numpy as np
import tensorflow as tf
from keras import layers, models
from keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix

from medmnist import OrganMNIST3D

# Carrega os splits padrão da MedMNIST
train_ds = OrganMNIST3D(split='train', download=True)
val_ds   = OrganMNIST3D(split='val',   download=True)
test_ds  = OrganMNIST3D(split='test',  download=True)

# Imagens e rótulos em numpy
x_train = train_ds.imgs       # (N_train, 28, 28, 28)
y_train = train_ds.labels     # (N_train, 1)
x_val   = val_ds.imgs
y_val   = val_ds.labels
x_test  = test_ds.imgs
y_test  = test_ds.labels

print("Shapes brutos:")
print("x_train:", x_train.shape, "y_train:", y_train.shape)
print("x_val:  ", x_val.shape,   "y_val:  ", y_val.shape)
print("x_test: ", x_test.shape,  "y_test: ", y_test.shape)


Shapes brutos:
x_train: (971, 28, 28, 28) y_train: (971, 1)
x_val:   (161, 28, 28, 28) y_val:   (161, 1)
x_test:  (610, 28, 28, 28) y_test:  (610, 1)


#Tratamento dos dados

##Normalização

In [7]:
# Normaliza para [0,1] e adiciona canal
x_train = (x_train.astype("float32") / 255.0)[..., np.newaxis]  # (N, 28,28,28,1)
x_val   = (x_val.astype("float32")   / 255.0)[..., np.newaxis]
x_test  = (x_test.astype("float32")  / 255.0)[..., np.newaxis]

print("x_train normalizado:", x_train.shape)


x_train normalizado: (971, 28, 28, 28, 1)


##One-hot encoding

In [8]:
# Tira a dimensão extra de rótulo (N,1) -> (N,)
y_train = y_train.squeeze()
y_val   = y_val.squeeze()
y_test  = y_test.squeeze()

num_classes = int(np.max(y_train) + 1)
print("Número de classes:", num_classes)

# One-hot
y_train_cat = to_categorical(y_train, num_classes)
y_val_cat   = to_categorical(y_val,   num_classes)
y_test_cat  = to_categorical(y_test,  num_classes)

print("y_train_cat:", y_train_cat.shape)


Número de classes: 11
y_train_cat: (971, 11)


##Datasets treinamento

In [9]:
batch_size = 16
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train_cat))
    .shuffle(buffer_size=len(x_train))
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

val_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_val, y_val_cat))
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_test, y_test_cat))
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)


#Criação da rede

In [10]:
def build_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    # Bloco 1
    x = layers.Conv3D(32, kernel_size=(3, 3, 3), padding="same", activation="relu")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool3D(pool_size=(2, 2, 2))(x)   # 28 -> 14

    # Bloco 2
    x = layers.Conv3D(64, kernel_size=(3, 3, 3), padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool3D(pool_size=(2, 2, 2))(x)   # 14 -> 7

    # Bloco 3
    x = layers.Conv3D(128, kernel_size=(3, 3, 3), padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)

    # GlobalAveragePooling3D em vez de Flatten (menos parâmetros)
    x = layers.GlobalAveragePooling3D()(x)

    # Regularização
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs, outputs, name="organmnist3d_cnn")

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

input_shape = x_train.shape[1:]  # (28,28,28,1)
model = build_model(input_shape, num_classes)
model.summary()


Model: "organmnist3d_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 28, 1)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d (Conv3D)                 │ (None, 28, 28, 28, 32) │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 28, 28, 28, 32) │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (None, 14, 14, 14, 32) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 14, 14, 14, 64) │        55,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 14, 14, 14, 64) │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (None, 7, 7, 7, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_2 (Conv3D)               │ (None, 7, 7, 7, 128)   │       221,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 7, 7, 7, 128)   │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling3d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling3D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 11)             │         1,419 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 279,883 (1.07 MB)

 Trainable params: 279,435 (1.07 MB)

 Non-trainable params: 448 (1.75 KB)

#Fit da rede

In [11]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=2,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=5,
    min_lr=1e-6
)

epochs = 60

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    callbacks=[early_stop, reduce_lr]
)


Epoch 1/60
61/61 ━━━━━━━━━━━━━━━━━━━━ 156s 2s/step - accuracy: 0.4177 - loss: 1.7076 - val_accuracy: 0.0932 - val_loss: 3.1219 - learning_rate: 0.0010
Epoch 2/60
61/61 ━━━━━━━━━━━━━━━━━━━━ 125s 2s/step - accuracy: 0.7556 - loss: 0.7867 - val_accuracy: 0.1801 - val_loss: 4.6585 - learning_rate: 0.0010
Epoch 3/60
61/61 ━━━━━━━━━━━━━━━━━━━━ 125s 2s/step - accuracy: 0.8087 - loss: 0.6174 - val_accuracy: 0.1615 - val_loss: 8.1333 - learning_rate: 0.0010
Epoch 4/60
61/61 ━━━━━━━━━━━━━━━━━━━━ 125s 2s/step - accuracy: 0.8403 - loss: 0.5254 - val_accuracy: 0.0932 - val_loss: 8.8864 - learning_rate: 0.0010
Epoch 5/60
61/61 ━━━━━━━━━━━━━━━━━━━━ 130s 2s/step - accuracy: 0.8605 - loss: 0.4467 - val_accuracy: 0.1739 - val_loss: 6.3422 - learning_rate: 0.0010
Epoch 6/60
61/61 ━━━━━━━━━━━━━━━━━━━━ 125s 2s/step - accuracy: 0.9100 - loss: 0.3541 - val_accuracy: 0.1801 - val_loss: 7.3011 - learning_rate: 0.0010
Epoch 7/60
61/61 ━━━━━━━━━━━━━━━━━━━━ 124s 2s/step - accuracy: 0.9336 - loss: 0.2558 - val_acc

#Avaliação do modelo

##Loss e acurácia em teste

In [12]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test loss: {test_loss:.4f}")
print(f"Test accuracy: {test_acc:.4f}")


39/39 ━━━━━━━━━━━━━━━━━━━━ 17s 423ms/step - accuracy: 0.9566 - loss: 0.1577
Test loss: 0.1582
Test accuracy: 0.9541


##F1-score, precision, recall, confusion matrix

In [13]:
# Probabilidades preditas
y_pred_probs = model.predict(test_dataset)
y_pred = np.argmax(y_pred_probs, axis=1)

# Rótulos verdadeiros (inteiros 0..C-1)
y_true = y_test

print(classification_report(y_true, y_pred, digits=4))

cm = confusion_matrix(y_true, y_pred)
print("Matriz de confusão:\n", cm)


39/39 ━━━━━━━━━━━━━━━━━━━━ 18s 461ms/step
              precision    recall  f1-score   support

           0     0.9583    1.0000    0.9787        69
           1     0.9836    0.8824    0.9302        68
           2     0.8750    0.9130    0.8936        69
           3     0.9538    0.9538    0.9538        65
           4     0.9385    0.9385    0.9385        65
           5     0.9692    0.9545    0.9618        66
           6     1.0000    0.9643    0.9818        28
           7     1.0000    1.0000    1.0000        21
           8     1.0000    1.0000    1.0000        21
           9     0.9577    0.9855    0.9714        69
          10     0.9571    0.9710    0.9640        69

    accuracy                         0.9541       610
   macro avg     0.9630    0.9603    0.9613       610
weighted avg     0.9549    0.9541    0.9540       610

Matriz de confusão:
 [[69  0  0  0  0  0  0  0  0  0  0]
 [ 0 60  7  0  0  1  0  0  0  0  0]
 [ 2  0 63  0  2  0  0  0  0  1  1]
 [ 0  0  0 62  2